<font size="+3"><strong> Wrangling Data with SQL</strong></font>

In [1]:
import sqlite3 as sql
import pandas as pd

# Prepare Data

## Connect

**Task 4.1.1:** connect to the `nepal.db` database. 


In [2]:
conn = sql.connect("data/nepal.db")

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///data/nepal.db

In [5]:
#!pip install jupysql --upgrade

## Explore

**Task 4.1.2:** Select all rows and columns from the `sqlite_schema` table, and examine the output. 


In [6]:
%%sql
select *
from sqlite_schema

*  sqlite:///data/nepal.db
Done.


type,name,tbl_name,rootpage,sql
table,id_map,id_map,54850,"CREATE TABLE ""id_map"" ( ""ward_id"" INTEGER, ""vdcmun_id"" INTEGER, ""vdcmun_name"" TEXT, ""district_id"" INTEGER, ""district_name"" TEXT, PRIMARY KEY(""ward_id""))"
table,mapping,mapping,709,"CREATE TABLE ""mapping"" ( ""individual_id"" INTEGER, ""household_id"" INTEGER, ""building_id"" INTEGER, PRIMARY KEY(""individual_id""))"
table,household_demographics,household_demographics,10175,"CREATE TABLE ""household_demographics"" ( ""household_id"" INTEGER, ""district_id"" INTEGER, ""vdcmun_id"" INTEGER, ""ward_id"" INTEGER, ""gender_household_head"" TEXT, ""age_household_head"" REAL, ""caste_household"" TEXT, ""education_level_household_head"" TEXT, ""income_level_household"" TEXT, ""size_household"" REAL, ""is_bank_account_present_in_household"" REAL, PRIMARY KEY(""household_id""))"
table,building_structure,building_structure,697,"CREATE TABLE ""building_structure"" ( ""building_id"" INTEGER, ""district_id"" INTEGER, ""vdcmun_id"" INTEGER, ""ward_id"" INTEGER, ""count_floors_pre_eq"" INTEGER, ""count_floors_post_eq"" INTEGER, ""age_building"" INTEGER, ""plinth_area_sq_ft"" INTEGER, ""height_ft_pre_eq"" INTEGER, ""height_ft_post_eq"" INTEGER, ""land_surface_condition"" TEXT, ""foundation_type"" TEXT, ""roof_type"" TEXT, ""ground_floor_type"" TEXT, ""other_floor_type"" TEXT, ""position"" TEXT, ""plan_configuration"" TEXT, ""has_superstructure_adobe_mud"" INTEGER, ""has_superstructure_mud_mortar_stone"" INTEGER, ""has_superstructure_stone_flag"" INTEGER, ""has_superstructure_cement_mortar_stone"" INTEGER, ""has_superstructure_mud_mortar_brick"" INTEGER, ""has_superstructure_cement_mortar_brick"" INTEGER, ""has_superstructure_timber"" INTEGER, ""has_superstructure_bamboo"" INTEGER, ""has_superstructure_rc_non_engineered"" INTEGER, ""has_superstructure_rc_engineered"" INTEGER, ""has_superstructure_other"" INTEGER, ""condition_post_eq"" TEXT, ""technical_solution_proposed"" TEXT, PRIMARY KEY(""building_id""))"
table,building_damage,building_damage,72162,"CREATE TABLE ""building_damage"" ( ""building_id"" INTEGER, ""district_id"" INTEGER, ""vdcmun_id"" INTEGER, ""ward_id"" INTEGER, ""damage_overall_collapse"" TEXT, ""damage_overall_leaning"" TEXT, ""damage_overall_adjacent_building_risk"" TEXT, ""damage_foundation_severe"" TEXT, ""damage_foundation_moderate"" TEXT, ""damage_foundation_insignificant"" TEXT, ""damage_roof_severe"" TEXT, ""damage_roof_moderate"" TEXT, ""damage_roof_insignificant"" TEXT, ""damage_corner_separation_severe"" TEXT, ""damage_corner_separation_moderate"" TEXT, ""damage_corner_separation_insignificant"" TEXT, ""damage_diagonal_cracking_severe"" TEXT, ""damage_diagonal_cracking_moderate"" TEXT, ""damage_diagonal_cracking_insignificant"" TEXT, ""damage_in_plane_failure_severe"" TEXT, ""damage_in_plane_failure_moderate"" TEXT, ""damage_in_plane_failure_insignificant"" TEXT, ""damage_out_of_plane_failure_severe"" TEXT, ""damage_out_of_plane_failure_moderate"" TEXT, ""damage_out_of_plane_failure_insignificant"" TEXT, ""damage_out_of_plane_failure_walls_ncfr_severe"" TEXT, ""damage_out_of_plane_failure_walls_ncfr_moderate"" TEXT, ""damage_out_of_plane_failure_walls_ncfr_insignificant"" TEXT, ""damage_gable_failure_severe"" TEXT, ""damage_gable_failure_moderate"" TEXT, ""damage_gable_failure_insignificant"" TEXT, ""damage_delamination_failure_severe"" TEXT, ""damage_delamination_failure_moderate"" TEXT, ""damage_delamination_failure_insignificant"" TEXT, ""damage_column_failure_severe"" TEXT, ""damage_column_failure_moderate"" TEXT, ""damage_column_failure_insignificant"" TEXT, ""damage_beam_failure_severe"" TEXT, ""damage_beam_failure_moderate"" TEXT, ""damage_beam_failure_insignificant"" TEXT, ""damage_infill_partition_failure_severe"" TEXT, ""damage_infill_partition_failure_moderate"" TEXT, ""damage_infill_partition_failure_insignificant"" TEXT, ""damage_staircase_severe"" TEXT, ""damage_staircase_moderate"" TEXT, ""damage_staircase_insignificant"" TEXT, "

**Task 4.1.3:** Select the `name` column from the `sqlite_schema` table, showing only rows where the **`type`** is `"table"`.

In [7]:
%%sql
SELECT name
FROM sqlite_schema
WHERE type="table"

*  sqlite:///data/nepal.db
Done.


name
id_map
mapping
household_demographics
building_structure
building_damage


**Task 4.1.4:** Select all columns from the `id_map` table, limiting your results to the first five rows. 


In [8]:
%%sql
SELECT *
FROM id_map
LIMIT 5

*  sqlite:///data/nepal.db
Done.


ward_id,vdcmun_id,vdcmun_name,district_id,district_name
120101,1201,Champadevi Rural Municipality,12,Okhaldhunga
120102,1201,Champadevi Rural Municipality,12,Okhaldhunga
120103,1201,Champadevi Rural Municipality,12,Okhaldhunga
120104,1201,Champadevi Rural Municipality,12,Okhaldhunga
120105,1201,Champadevi Rural Municipality,12,Okhaldhunga


**Task 4.1.5:** How many observations are in the `id_map` table? Use the `count` command to find out.


In [9]:
%%sql
SELECT count(*)
FROM id_map

*  sqlite:///data/nepal.db
Done.


count(*)
949


**Task 4.1.6:** What districts are represented in the `id_map` table? Use the `distinct` command to determine the unique values in the **`district_id`** column.


In [10]:
%config SqlMagic.displaylimit = None
%sql SELECT distinct(district_id) FROM id_map

displaylimit: Value None will be treated as 0 (no limit)
*  sqlite:///data/nepal.db
Done.


district_id
12
20
21
22
23
24
28
29
30
31


**Task 4.1.8:** For our model, we'll focus on Gorkha (district `36`). Select all columns that from `id_map`, showing only rows where the **`district_id`** is `36` and limiting your results to the first five rows.


In [11]:
%%sql
SELECT *
FROM id_map
WHERE district_id = 36
LIMIT 5

*  sqlite:///data/nepal.db
Done.


ward_id,vdcmun_id,vdcmun_name,district_id,district_name
360101,3601,Aarughat Rural Municipality,36,Gorkha
360102,3601,Aarughat Rural Municipality,36,Gorkha
360103,3601,Aarughat Rural Municipality,36,Gorkha
360104,3601,Aarughat Rural Municipality,36,Gorkha
360105,3601,Aarughat Rural Municipality,36,Gorkha


**Task 4.1.11:** Select all the columns from the `building_structure` table, and limit your results to the first five rows. 


In [12]:
%%sql
SELECT *
FROM building_structure
LIMIT 5

*  sqlite:///data/nepal.db
Done.


building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,technical_solution_proposed
120101000011,12,1207,120703,1,1,9,288,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Used in risk,Major repair
120101000021,12,1207,120703,1,1,15,364,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Repaired and used,Reconstruction
120101000031,12,1207,120703,1,1,20,384,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Minor repair
120101000041,12,1207,120703,1,1,20,312,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Minor repair
120101000051,12,1207,120703,1,1,30,308,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Minor repair


**Task 4.1.12:** How many building are there in the `building_structure` table? Use the `count` command to find out.


In [13]:
%%sql
SELECT count(*)
FROM building_structure

*  sqlite:///data/nepal.db
Done.


count(*)
762106


In [14]:
%%sql
SELECT *
FROM building_damage
LIMIT 5

*  sqlite:///data/nepal.db
Done.


building_id,district_id,vdcmun_id,ward_id,damage_overall_collapse,damage_overall_leaning,damage_overall_adjacent_building_risk,damage_foundation_severe,damage_foundation_moderate,damage_foundation_insignificant,damage_roof_severe,damage_roof_moderate,damage_roof_insignificant,damage_corner_separation_severe,damage_corner_separation_moderate,damage_corner_separation_insignificant,damage_diagonal_cracking_severe,damage_diagonal_cracking_moderate,damage_diagonal_cracking_insignificant,damage_in_plane_failure_severe,damage_in_plane_failure_moderate,damage_in_plane_failure_insignificant,damage_out_of_plane_failure_severe,damage_out_of_plane_failure_moderate,damage_out_of_plane_failure_insignificant,damage_out_of_plane_failure_walls_ncfr_severe,damage_out_of_plane_failure_walls_ncfr_moderate,damage_out_of_plane_failure_walls_ncfr_insignificant,damage_gable_failure_severe,damage_gable_failure_moderate,damage_gable_failure_insignificant,damage_delamination_failure_severe,damage_delamination_failure_moderate,damage_delamination_failure_insignificant,damage_column_failure_severe,damage_column_failure_moderate,damage_column_failure_insignificant,damage_beam_failure_severe,damage_beam_failure_moderate,damage_beam_failure_insignificant,damage_infill_partition_failure_severe,damage_infill_partition_failure_moderate,damage_infill_partition_failure_insignificant,damage_staircase_severe,damage_staircase_moderate,damage_staircase_insignificant,damage_parapet_severe,damage_parapet_moderate,damage_parapet_insignificant,damage_cladding_glazing_severe,damage_cladding_glazing_moderate,damage_cladding_glazing_insignificant,area_assesed,damage_grade,technical_solution_proposed,has_repair_started,has_damage_foundation,has_damage_roof,has_damage_corner_separation,has_damage_diagonal_cracking,has_damage_in_plane_failure,has_damage_out_of_plane_failure,has_damage_out_of_plane_walls_ncfr_failure,has_damage_gable_failure,has_damage_delamination_failure,has_damage_column_failure,has_damage_beam_failure,has_damage_infill_partition_failure,has_damage_staircase,has_damage_parapet,has_damage_cladding_glazing,has_geotechnical_risk,has_geotechnical_risk_land_settlement,has_geotechnical_risk_fault_crack,has_geotechnical_risk_liquefaction,has_geotechnical_risk_landslide,has_geotechnical_risk_rock_fall,has_geotechnical_risk_flood,has_geotechnical_risk_other
120101000011,12,1207,120703,Moderate-Heavy,Insignificant/light,None,None,Moderate-Heavy-(<1/3),Insignificant/light-(<1/3),Severe-Extreme-(<1/3),None,Insignificant/light-(>2/3),Severe-Extreme-(>2/3),None,None,Severe-Extreme-(<1/3),None,None,Severe-Extreme-(>2/3),None,None,None,Moderate-Heavy-(>2/3),None,Severe-Extreme-(>2/3),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Both,Grade 3,Major repair,0.0,None,1.0,1.0,1.0,1.0,None,1.0,0.0,0.0,None,None,None,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
120101000021,12,1207,120703,Severe-Extreme,Severe-Extreme,Insignificant/light,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,Severe-Extreme-(>2/3),None,None,None,None,None,Severe-Extreme-(>2/3),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Exterior,Grade 5,Reconstruction,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,None,None,None,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
120101000031,12,1207,120703,Moderate-Heavy,Moderate-Heavy,Moderate-Heavy,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,Moderate-Heavy-(>2/3),None,None,None,Insignificant/light-(1/3-2/3),None,None,None,None,None,None,None,None,None,N/A,None,None,None,None,None,None,None,None,Both,Grade 2,Minor repair,1.0,None,None,None,None,None,Non

**Task 4.1.15:**  include the **`damage_grade`** column to your table by adding a second `JOIN` for the `building_damage` table. Be sure to limit your results to the first five rows of the new table.


In [15]:
%%sql
SELECT s.* ,d.damage_grade
FROM building_structure as s 
JOIN building_damage AS d
ON s.building_id = d.building_id
WHERE s.district_id=36
LIMIT 5

*  sqlite:///data/nepal.db
Done.


building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,technical_solution_proposed,damage_grade
360101000011,36,3609,360903,1,0,40,336,6,0,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Rubble unclear,Reconstruction,Grade 5
360101000021,36,3609,360903,2,2,12,420,13,13,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Used in risk,Minor repair,Grade 3
360101000031,36,3609,360903,3,3,20,560,18,18,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,Minor repair,Grade 2
360101000041,36,3609,360903,2,2,15,390,12,12,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Used in risk,Reconstruction,Grade 4
360101000051,36,3609,360903,2,2,15,290,13,13,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Square,0,1,0,0,0,0,0,0,0,0,0,Damaged-Not used,Reconstruction,Grade 4


## Import

**Task 4.1.17:** Put your last SQL query into a string and assign it to the variable `query`.

In [16]:
query = """
SELECT s.* ,d.damage_grade
FROM building_structure as s 
JOIN building_damage AS d
ON s.building_id = d.building_id
WHERE s.district_id=36
"""

**Task 4.1.18:** Use the [`read_sql`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html#pandas-read-sql) from the pandas library to create a DataFrame from your `query`. Be sure that the **`building_id`** is set as your index column.


In [17]:
df = pd.read_sql(query,conn,index_col="building_id")

df.head()

,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,...,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,technical_solution_proposed,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
360101000011,36,3609,360903,1,0,40,336,6,0,Flat,...,0,0,0,0,0,0,0,Damaged-Rubble unclear,Reconstruction,Grade 5
360101000021,36,3609,360903,2,2,12,420,13,13,Flat,...,0,0,0,0,0,0,0,Damaged-Used in risk,Minor repair,Grade 3
360101000031,36,3609,360903,3,3,20,560,18,18,Flat,...,0,0,0,0,0,0,0,Damaged-Repaired and used,Minor repair,Grade 2
360101000041,36,3609,360903,2,2,15,390,12,12,Flat,...,0,0,0,0,0,0,0,Damaged-Used in risk,Reconstruction,Grade 4
360101000051,36,3609,360903,2,2,15,290,13,13,Flat,...,0,0,0,0,0,0,0,Damaged-Not used,Reconstruction,Grade 4


In [18]:
df.shape

(78074, 30)